In [ ]:
!wget https://github.com/endgameinc/malware_evasion_competition/blob/master/models/malconv/malconv.checkpoint?raw=true -O malconv.checkpoint

In [ ]:
#　サンプルファイルのダウンロード
!wget https://github.com/InQuest/malware-samples/blob/master/2019-02-Trickbot/374ef83de2b254c4970b830bb93a1dd79955945d24b824a0b35636e14355fe05?raw=true -O sample.bin
!wget https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe -O putty.exe

In [ ]:
# liefの最新版だと、Colabがクラッシュすることがあるので 0.10.0を使用する
!pip install lief==0.10.0

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# PyTorchでは、torch.nn.Moduleクラスを継承して独自のネットワークを作成する
class MalConv(nn.Module):
    # ニューラルネットワークの層を定義する
    def __init__(self, out_size=2, channels=128, \
            window_size=512, embd_size=8):
        # nn.Module内の初期化関数を実行する        
        super(MalConv, self).__init__()

        # 埋め込み層。各入力バイトを8次元ベクトルにマッピングする
        self.embd = nn.Embedding(257, embd_size, padding_idx=0)
        
        # 一次元の畳み込み層2つ
        self.window_size = window_size
        self.conv_1 = nn.Conv1d(embd_size, channels, 
            window_size, stride=window_size, bias=True)
        self.conv_2 = nn.Conv1d(embd_size, channels, 
            window_size, stride=window_size, bias=True)
        
        # プーリング層
        self.pooling = nn.AdaptiveMaxPool1d(1)
        
        # 全結合層2つ
        self.fc_1 = nn.Linear(channels, channels)
        self.fc_2 = nn.Linear(channels, out_size)
    
    # 層間の計算を定義する
    def forward(self, x):
        # 入力を埋め込み層に与えた結果を得る
        x = self.embd(x.long())
        x = torch.transpose(x,-1,-2) # 行列の転置
        
        # 畳み込み層1の結果を得る
        cnn_value = self.conv_1(x)

        # 畳み込み層2の結果を取得し、シグモイド関数に掛ける
        gating_weight = torch.sigmoid(self.conv_2(x))

        # 積をとる
        x = cnn_value * gating_weight
        
        # プーリング層の結果を得る
        x = self.pooling(x)
        
        x = x.view(x.size(0), -1) # 平滑化
        
        # 全結合層1の結果をReLU関数に掛ける
        x = F.relu(self.fc_1(x))

        # 全結合層2の結果を得る
        x = self.fc_2(x)
        
        return x

In [ ]:
MALCONV_MODEL_PATH = 'malconv.checkpoint'

class MalConvModel(object):
    def __init__(self, model_path, thresh=0.5, name='malconv'): 
        # MalConvのロード
        self.model = MalConv(channels=256, 
            window_size=512, embd_size=8).train()
        # 学習済のモデルをロードする
        weights = torch.load(model_path,map_location='cpu')
        self.model.load_state_dict(weights['model_state_dict'])
        self.thresh = thresh
        self.__name__ = name

    def predict(self, bytez):
        # ファイルのバイト列を整数（0～255）に変換する
        _inp = torch.from_numpy( \
            np.frombuffer(bytez,dtype=np.uint8)[np.newaxis,:])
        # MalConvの結果をソフトマックス関数に掛け、マルウェアらしさの確率を出力する
        with torch.no_grad():
            outputs = F.softmax(self.model(_inp), dim=-1)

        # マルウェアらしさが閾値0.5を越えていたらマルウェアと判定する
        return outputs.detach().numpy()[0,1] > self.thresh

In [ ]:
# ファイルのバイト列を取得する
with open('sample.bin', 'rb') as f:
    bytez = f.read()

# MalConvModelを用いて分類する
malconv = MalConvModel(MALCONV_MODEL_PATH, thresh=0.5)
print(f'{malconv.__name__}:  {malconv.predict(bytez)}')

In [ ]:
!pip install pefile

In [ ]:
from pefile import *

# PEインスタンスを作成する
pe = PE('sample.bin')

# 生のバイト列にはメモリマップドファイルとしてアクセスできる
# ここでは、DOSヘッダの冒頭に含まれる文字列「MZ」を取得する
print(pe.__data__[0:2])

# PEヘッダとそのメンバにはFILE_HEADERからアクセスできる
print('NumberOfSection is {0}'.format(pe.FILE_HEADER.NumberOfSections))

# オプショナルヘッダとそのメンバにはOPTIONAL_HEADERからアクセスできる
print('AddressOfEntryPoint at 0x{0:08x}'.format(pe.OPTIONAL_HEADER.AddressOfEntryPoint))

# データディレクトリはそれぞれDIRECTORY_ENTRY_IMPORT、
# DIRECTORY_ENTRY_DEBUGなどからアクセスできる
# ここでは、インポート関数のデータディレクトリから
# インポートしているDLLとそのAPIを列挙する
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    dll_name = entry.dll.decode('utf-8')
    print(dll_name)
    for func in entry.imports:
        try:
            print('\t{0} at 0x{1:08x}'.format(func.name.decode('utf-8'), func.address))
        except:
            pass
        
# セクションテーブルは各セクションのインスタンスとしてアクセスできる
# ここでは、各セクションの名前とPointerToRawDataを列挙する
# また、各セクションのデータをget_dataメソッドを通じて取得する
for section in pe.sections:
    print(section.Name.decode('utf-8'))
    print('\tPointerToRawData at 0x{0:08x}'.format(section.PointerToRawData))
    print('\t{0} ...'.format(section.get_data()[0:10]))

In [ ]:
import datetime

class PEManipulator(PE):
    def __init__(self, name=None, data=None, fast_load=None):
        super(PEManipulator, self).__init__(name, data, fast_load)

    def reset_timestamp(self, new_timestamp_str=None):
        # 指定された日時の文字列をUNIX時間に変換する
        if new_timestamp_str == None:
            new_timestamp_str = datetime.datetime.now().strftime( \
                '%Y-%m-%d %H:%M:%S')
        new_timestamp = int(time.mktime(time.strptime( \
                new_timestamp_str, '%Y-%m-%d %H:%M:%S')))
        # 当該UNIX時間でTimeDateStampを上書きする
        self.FILE_HEADER.TimeDateStamp = new_timestamp 
        return

pe = PEManipulator('sample.bin')
pe.reset_timestamp(new_timestamp_str='2020-01-01 00:00:00')
pe.write('sample_modified.bin')

In [ ]:
import random

class PEManipulator(PE):
    def __init__(self, name=None, data=None, fast_load=None):
        super(PEManipulator, self).__init__(name, data, fast_load)
        # コードケイブのアドレスとサイズを記録する辞書を初期化する
        self.code_cave_dict = {}
    
    def reset_timestamp(self, new_timestamp_str=None):
        # 指定された日時の文字列をUNIX時間に変換する
        if new_timestamp_str == None:
            new_timestamp_str = datetime.datetime.now().strftime( \
                '%Y-%m-%d %H:%M:%S')
        new_timestamp = int(time.mktime(time.strptime( \
                new_timestamp_str, '%Y-%m-%d %H:%M:%S')))
        # 当該UNIX時間でTimeDateStampを上書きする
        self.FILE_HEADER.TimeDateStamp = new_timestamp 
        return

    def add_overlay(self, upper=255, overlay=None):
        # 書き込むサイズを設定する
        L = 2**random.randint(5, 8)
        # 追加データを指定しない場合、
        # 指定された文字コードとサイズの範囲でランダムなデータを生成する
        if overlay == None:
            overlay = bytes([random.randint(0, upper) for _ in range(L)])
        # ファイルの末尾にデータを追記する
        self.__data__ = (self.__data__[:-1] + overlay)
        
        return

    def find_code_cave(self, min_cave_size=100):
        # セクションごとにコードケイブを探索する
        for section in self.sections:
            code_cave_offset = 0
            code_cave_size = 0

            # セクションに含まれるデータを取得する
            data = section.get_data()

            for i, byte in enumerate(data):
                code_cave_offset += 1

                # 現在のデータが0x00ならコードケイブの候補とする
                if byte == 0x00:
                    code_cave_size += 1
                    continue

                # コードケイブの候補が一定のサイズ以上連続している場合、
                elif code_cave_size > min_cave_size:
                    # ヒューリスティック：コードケイブの候補間に
                    # わずかなデータがあればコードケイブではないと判断し、
                    # そうでなければコードケイブであると判断する
                    if i < len(data)-1 and data[i+1] == 0x00:
                        break
                    
                    # ファイル上のコードケイブの起点となるアドレスを取得し、
                    code_cave_address = section.PointerToRawData \
                            + code_cave_offset - code_cave_size - 1
                    # コードケイブの起点アドレスとコードケイブのサイズを登録する
                    self.code_cave_dict.update({code_cave_address:
                            code_cave_size})

                code_cave_size = 0

        return

    def add_code_cave(self, upper=255, min_cave_size=100, code_cave=None):
        # コードケイブ辞書が空の場合、コードケイブを探索する
        if self.code_cave_dict == {}: self.find_code_cave(min_cave_size)
        try:
            # コードケイブ辞書からランダムに1件取得する
            code_cave_address, code_cave_size = \
                random.choice(list(self.code_cave_dict.items()))
            L = code_cave_size
            # 追加データを指定しない場合、
            # 指定された文字コードとサイズの範囲でランダムなデータを生成する
            if code_cave == None:
                code_cave = bytes([random.randint(0, upper) \
                    for _ in range(L)])
            # 追加データを指定する場合、
            # サイズの範囲に収まるように切り詰める
            else:
                code_cave = code_cave[0:L]
            # 起点となるアドレスからコードケイブを追記する
            self.set_bytes_at_offset(code_cave_address, code_cave)
        except: # 辞書が空の場合は何もしない
            pass

        return

pe = PEManipulator('sample.bin')
pe.add_overlay()
pe.add_code_cave()
pe.write('sample_modified.exe')

In [ ]:
!pip install keras-rl2

In [ ]:
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

In [ ]:
# Start virtual display
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display)

In [ ]:
# OpenAI Gymをインポートする
import gym

# KerasおよびKeras-RLをインポートする
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

# 一部環境でのエラー抑制用イディオム
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# 強化学習タスクの環境を初期化する
ENV_NAME = 'CartPole-v0'
env = gym.make(ENV_NAME)
nb_actions = env.action_space.n

# Q関数の近似に用いるニューラルネットワークを定義する
# 状態空間の次元env.observation_space.shapeを入力、
# 行動空間の次元nb_actionsを出力としていれば、中間層は好きに積み重ねてよい
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

# Experience Replay用のメモリを用意する
memory = SequentialMemory(limit=50000, window_length=1)

# 行動ポリシーとして確率1-epsでランダムな行動をとらせるようにする
policy = EpsGreedyQPolicy(eps=.1)

# エージェントを初期化する
# DQNAgentの引数にはさきほど定義したネットワーク、行動空間の次元数、
# Exprerience Replay用のメモリ、割引率、Target Q-Networkのアップデート頻度、
# 行動ポリシーなどを指定する
dqn = DQNAgent(model=model, nb_actions=nb_actions, gamma=0.99, memory=memory, nb_steps_warmup=100,target_model_update=1e-2, policy=policy)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mse'])

# エージェントを学習させる
# ここでは100,000回の行動を通じてQ関数を近似している
history = dqn.fit(env=env, nb_steps=100000, visualize=True, verbose=2)

# ニューラルネットワークの重みを保存する
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# 学習済のエージェントをテストする
# ここでは5回のエピソードにわたってエージェントをテストしている
dqn.test(env=env, nb_episodes=5, visualize=True)

# 学習過程をプロットするためにmatplotlibをインポートする
%matplotlib inline
import matplotlib.pyplot as plt

# 学習過程の履歴を取得する
nb_episode_steps = history.history['nb_episode_steps']
episode_reward = history.history['episode_reward']

# 取得した学習過程をプロットする
plt.subplot(2,1,1)
plt.plot(nb_episode_steps)
plt.ylabel('step')

plt.subplot(2,1,2)
plt.plot(episode_reward)
plt.xlabel('episode')
plt.ylabel('reward')

plt.show()

In [ ]:
!pip install git+https://github.com/elastic/ember.git

In [ ]:
import numpy as np
import gym
import gym.spaces
from ember import PEFeatureExtractor

class MalwareEvasionEnv(gym.Env):
    def __init__(self):
        super().__init__()
        # 行動空間を定義する（必須）
        # ここではreset_timestamp, add_overlay,
        # add_code_cave, そしてadd_fake_importsの4つ 
        self.action_space = gym.spaces.Discrete(4)
        # 報酬の最小値と最大値を定義する（必須）
        self.reward_range = [-1., 100.]
        # 環境を初期化する（必須）
        self.reset()

    # 環境を初期化する（必須）
    # 戻り値は初期状態
    def reset(self):
        # マルウェアのバイト列を読み取り、
        with open('sample.bin', 'rb') as f:
            malware_bytez = f.read()        
        self.bytez = malware_bytez
        # 4章で用いたEMBERの特徴抽出器を使って状態を生成する
        # この特徴抽出の内部ではLIEFが用いられているが、
        # pefileでも同様の処理は可能である
        self.extractor = PEFeatureExtractor(2)
        self.observation_space = np.array( \
            self.extractor.feature_vector(self.bytez), \
                dtype=np.float32)
        # 分類器を初期化する
        self.model = MalConvModel(MALCONV_MODEL_PATH, thresh=0.5)
        return self.observation_space

    # 行動を実行する（必須）
    # 戻り値は次状態、報酬、エピソード終了フラグ、追加情報 
    def step(self, action):
        # エージェントが0, 1, 2, 3のどれかをactionとして渡してくるので、
        # 愚直にPEManipulatorのメソッドと対応づけて実行する
        pe = PEManipulator(data=self.bytez)
        if action == 0:
            pe.reset_timestamp()
        if action == 1:
            pe.add_overlay()
        if action == 2:
            pe.add_code_cave()
        if action == 3:
            pe.add_fake_imports()
        
        # 状態を更新する
        self.bytez = pe.write()
        self.observation_space = np.array( \
            self.extractor.feature_vector(self.bytez), \
                dtype=np.float32)
        
        # 報酬を取得する
        # ここではMalConvがマルウェアだと判定すると-1、
        # MalConvが良性ファイルだと判定すると100の報酬を与える
        reward = -1 if self.model.predict(self.bytez) else 100
        
        # 報酬をもとにエピソード終了フラグを更新する
        episode_over = False if reward == -1 else True
        
        return self.observation_space, reward, episode_over, {}

    # 環境を可視化する（必須）
    # 中身は空でよい
    def render(self, mode='human', close=False):
        pass

    # 環境を閉じる
    # 中身は空でよい
    def _close(self):
        pass

    # シードを固定する
    # 中身は空でよい
    def _seed(self, seed=None):
        pass

In [ ]:
from gym.envs.registration import register

register(
    # 環境のIDを登録する
    # IDは<環境名>-v<バージョン番号>というフォーマットに沿って記述する
    id='MalwareEvasionEnv-v0',
    entry_point='__main__:MalwareEvasionEnv'
    # エントリポイントを定義する
    # エントリポイントは<名前空間>:<クラス名>というフォーマットに沿って記述する
    # ここではJupyter NotebookまたはGoogle Colaboratoryのセル中に環境が存在することを前提としている
)

In [ ]:
class MalConvModel(object):
    def __init__(self, model_path, thresh=0.5, name='malconv'): 
    # MalConvのロード
        self.model = MalConv(channels=256, 
        window_size=512, embd_size=8).train()
    # 学習済のモデルをロードする
        weights = torch.load(model_path,map_location='cpu')
        self.model.load_state_dict(weights['model_state_dict'])
        self.thresh = thresh
        self.__name__ = name

    def predict_with_score(self, bytez):
        # ファイルのバイト列を整数（0～255）に変換する
        _inp = torch.from_numpy( \
            np.frombuffer(bytez,dtype=np.uint8)[np.newaxis,:])
        # MalConvの結果をソフトマックス関数に掛け、マルウェアらしさの確率を出力する
        with torch.no_grad():
            outputs = F.softmax(self.model(_inp), dim=-1)

        # スコアを直接返す
        return outputs.detach().numpy()[0,1]

class MalwareEvasionEnv(gym.Env):
    def __init__(self):
        super().__init__()
        # 行動空間を定義する（必須）
        # ここでは以下の13件の行動を用いる
        # - reset_timestamp(現在日時)
        # - add_overlay(ランダムに生成したデータ)
        # - add_overlay(良性ファイルのデータ片)を5通り
        # - add_code_cave(ランダムに生成したデータ)
        # - add_code_cave(良性ファイルのデータ片)を5通り
        self.action_space = gym.spaces.Discrete(13)
        # 報酬の最小値と最大値を定義する（必須）
        self.reward_range = [-1., 100.]
        # 環境を初期化する（必須）
        self.reset()

    # 環境を初期化する（必須）
    # 戻り値は初期状態
    def reset(self):        
        # マルウェアのバイト列を読み取り、
        with open('sample.bin', 'rb') as f:
            malware_bytez = f.read()
        self.bytez = malware_bytez
        # 4章で用いたEMBERの特徴抽出器を使って状態を生成する
        # この特徴抽出の内部ではLIEFが用いられているが、
        # pefileでも同様の処理は可能である
        self.extractor = PEFeatureExtractor(2)
        self.observation_space = np.array( \
            self.extractor.feature_vector(self.bytez), \
                dtype=np.float32)
        
        # 良性ファイルのバイト列を読み取り、
        with open('putty.exe', 'rb') as f:
            benign_bytez = f.read()
        # 分割したデータのリストを作成する
        num_splits = 5
        offset = int(len(benign_bytez) / num_splits)
        self.benign_bytez_list = \
            [benign_bytez[i: i+offset] \
                for i in range(0, len(benign_bytez), offset)]
        
        # 分類器を初期化する
        self.model = MalConvModel(MALCONV_MODEL_PATH, thresh=0.5)
        
        # スコアを初期化する
        self.prev_score = 1
        self.thresh = 0.5
        self.steps = 0
        self.steps_limit = 100
        
        return self.observation_space

  # 行動を実行する（必須）
    # 戻り値は次状態、報酬、エピソード終了フラグ、追加情報 
    def step(self, action):
        # エージェントが0〜12のいずれかをactionとして渡してくるので、
        # PEManipulatorのメソッドと対応づけて実行する
        pe = PEManipulator(data=self.bytez)
        if action == 0:
            pe.reset_timestamp()
        if action == 1:
            pe.add_overlay()
        # 良性ファイルの一部をオーバーレイとして追加する
        if action in range(2, 6):
            pe.add_overlay( \
                overlay=self.benign_bytez_list[action-2])
        if action == 7:
            pe.add_code_cave()
        # 良性ファイルの一部をコードケイブとして追加する
        if action in range(8, 12):
            pe.add_code_cave( \
                code_cave=self.benign_bytez_list[action-8])
        
        # 状態を更新する
        self.bytez = pe.write()
        self.observation_space = np.array( \
            self.extractor.feature_vector(self.bytez), \
                dtype=np.float32)

        # 報酬を取得する
        # ここではMalConvの悪性スコアが
        # 前回のスコアより低くなれば+1、低くならなければ-1、
        # そしてMalConvが良性ファイルだと判定すると100の報酬を与える
        # 報酬または行動回数に応じてエピソード終了フラグも更新する
        episode_over = False
        self.steps += 1

        score = self.model.predict_with_score(self.bytez)
        if score < self.prev_score:
            reward = 1
        if score < self.thresh:
            reward = 100
            episode_over = True
        else:
            reward = -1

        self.prev_score = score

        if self.steps > self.steps_limit:
            episode_over = True
        
        # 報酬をもとにエピソード終了フラグを更新する
        episode_over = False if reward == -1 else True

        return self.observation_space, reward, episode_over, {}

    # 環境を可視化する（必須）
    # 中身は空でよい
    def render(self, mode='human', close=False):
        pass

    # 環境を閉じる
    # 中身は空でよい
    def _close(self):
        pass

    # シードを固定する
    # 中身は空でよい
    def _seed(self, seed=None):
        pass

In [ ]:
from gym.envs.registration import register

register(
    # 環境のIDを登録する
    # IDは<環境名>-v<バージョン番号>というフォーマットに沿って記述する
    id='MalwareEvasionEnv-v1',
    entry_point='__main__:MalwareEvasionEnv'
    # エントリポイントを定義する
    # エントリポイントは<名前空間>:<クラス名>というフォーマットに沿って記述する
    # ここではJupyter NotebookまたはGoogle Colaboratoryのセル中に環境が存在することを前提としている
)

In [ ]:
# OpenAI Gymをインポートする
import gym

# KerasおよびKeras-RLをインポートする
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

# 一部環境でのエラー抑制用イディオム
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# 強化学習タスクの環境を初期化する
ENV_NAME = 'MalwareEvasionEnv-v1'
env = gym.make(ENV_NAME)
nb_actions = env.action_space.n

# Q関数の近似に用いるニューラルネットワークを定義する
# 状態空間の次元env.observation_space.shapeを入力、
# 行動空間の次元nb_actionsを出力としていれば、中間層は好きに積み重ねてよい
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

# Experience Replay用のメモリを用意する
memory = SequentialMemory(limit=50000, window_length=1)

# 行動ポリシーとして確率1-epsでランダムな行動をとらせるようにする
policy = EpsGreedyQPolicy(eps=.1)

# エージェントを初期化する
# DQNAgentの引数にはさきほど定義したネットワーク、行動空間の次元数、
# Exprerience Replay用のメモリ、割引率、Target Q-Networkのアップデート頻度、
# 行動ポリシーなどを指定する
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory,
        gamma=0.99, target_model_update=1e-2, policy=policy)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mse'])

# エージェントを学習させる
# ここでは100,000回の行動を通じてQ関数を近似している
history = dqn.fit(env=env, nb_steps=100000, visualize=True, verbose=2)

# ニューラルネットワークの重みを保存する
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# 学習済のエージェントをテストする
# ここでは5回のエピソードにわたってエージェントをテストしている
dqn.test(env=env, nb_episodes=5, visualize=True)

# 学習過程をプロットするためにmatplotlibをインポートする
%matplotlib inline
import matplotlib.pyplot as plt

# 学習過程の履歴を取得する
nb_episode_steps = history.history['nb_episode_steps']
episode_reward = history.history['episode_reward']

# 取得した学習過程をプロットする
plt.subplot(2,1,1)
plt.plot(nb_episode_steps)
plt.ylabel('step')

plt.subplot(2,1,2)
plt.plot(episode_reward)
plt.xlabel('episode')
plt.ylabel('reward')

plt.show()